In [1]:
#conda install -c conda-forge kaggle
#conda install -c anaconda openpyxl
import scipy

In [2]:
#pip install --upgrade scipy==1.5.3


In [3]:
import pandas as pd
import kaggle as kg

import warnings
warnings.filterwarnings('ignore')

In [4]:
# Download do dataset do Kaggle
kg.api.authenticate()
kg.api.dataset_download_files(dataset="Sírio-Libanes/covid19", path='gt.zip', unzip=True)

In [5]:
# Lendo o arquivo
dados = pd.read_excel('gt.zip/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx', engine='openpyxl')
dados

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,...,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-6,0
3,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.000000,-1.000000,NaN,NaN,NaN,NaN,-1.000000,-1.000000,6-12,0
4,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,...,-0.238095,-0.818182,-0.389967,0.407558,-0.230462,0.096774,-0.242282,-0.814433,ABOVE_12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,384,0,50th,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1921,384,0,50th,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
1922,384,0,50th,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,4-6,0
1923,384,0,50th,1,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,6-12,0


In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
x_columns = dados.describe().columns

dados_limpos = dados.dropna()

In [8]:
y = dados_limpos["ICU"]
x = dados_limpos[x_columns].drop(["ICU"], axis=1)

In [9]:
modelo = LogisticRegression()
modelo.fit(x, y)

LogisticRegression()

In [10]:
y

4       1
6       1
8       1
9       1
14      1
       ..
1904    1
1914    1
1919    0
1921    0
1924    0
Name: ICU, Length: 545, dtype: int64

In [11]:
sum(modelo.predict(x) == y)

496

In [12]:
sum(modelo.predict(x) != y)


49

In [13]:
accuracy = sum(modelo.predict(x) == y) / len(y) * 100
accuracy

91.00917431192661

In [17]:
# chuta de acordo com o % de casos
from sklearn.dummy import DummyClassifier

In [16]:
modelo = DummyClassifier()
modelo.fit(x, y)
sum(modelo.predict(x) == y) / len(y) * 100

49.357798165137616

In [19]:
from sklearn.metrics import accuracy_score

In [20]:
accuracy_score(y, modelo.predict(x))

0.5082568807339449

In [43]:
import numpy as np
from sklearn.model_selection import train_test_split

In [44]:
np.random.seed(73246)

In [45]:
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y)

In [46]:
print(len(x_train), len(x_test))
print(len(y_train), len(y_test))

408 137
408 137


In [47]:
modelo = LogisticRegression()
modelo.fit(x_train, y_train)

y_prediction = modelo.predict(x_test)
accuracy_score(y_test, y_prediction)

0.8248175182481752

# REFERÊNCIAS

<b> <p>&#128218; REFERÊNCIAS:</p></b>  


- <b> Medium: </b> <a href="https://medium.com/@jeff.daniel77/accessing-the-kaggle-com-api-with-jupyter-notebook-on-windows-d6f330bc6953">Accessing the Kaggle.com API with Jupyter Notebook on Windows</a> 
<br>

- <b> Stack Overflow: </b> <a href="https://stackoverflow.com/questions/57051372/how-to-read-file-from-kaggle-in-jupyter-notebook-in-microsoft-azure">How to read file from Kaggle in Jupyter Notebook in Microsoft Azure?</a>
<br>

- <b> W3 Schools: </b> <a href="https://www.w3schools.com/charsets/ref_emoji.asp">Emoji Unicode Reference</a>
<br>

- <b> Kaggle: </b> <a href="https://www.kaggle.com/S%C3%ADrio-Libanes/covid19">COVID-19 - Clinical Data to assess diagnosis</a>